# CH01.Why we have to use container

### 1. 배경
 가상환경이란 특정 공간에서만 사용하도록 제한된 공간을 만드는 것이다. 이렇게 함으로써 프로젝트 별로 다른 모듈의 버전을 유지하고 관리할 수 있다. 추가적으로 global dependency가 되는 것을 막을 수 있다.<br>
 여러명의 클라이언트가 작업을 하게 되었을 상황에서는 프로젝트에 따라 각각 다른 버전이 필요하다. 이 때 글로벌하게 하나의 환경에 설치하는 것은 번거로움을 유발하고 비효율적이다. 1번 클라이언트 쪽의 프로젝트는 1번 버전, 2번 클라이언트 쪽의 프로젝트는 2번 버전을 사용하게 되면, 이 때 마다 한쪽 버전을 지우고 다시 설치하는 번거로움이 발생한다. 이러한 번거로움과 꼬임을 제거해주는 것이 가상환경이다.


### 2. VMware, VBox
Hypervisor 기술을 사용하여 하드웨어를 추상화하는 가상머신들이다. Hypervisor 기술이란 호스트 컴퓨터에서 다수의 운영 체제를 동시에 실행하기 위한 논리적 플랫폼이고 가상화 머신 모니터 또는 가상화 머신 매니저, VMM이라고 부른다. Hypervisor는 일반적으로 2가지로 나뉜다. Type1(native 또는 bare-metal)의 경우는 운영 체제가 프로그램을 제어하듯이 하이퍼바이저가 해당 하드웨어에서 직접 실행되며 게스트 운영 체제는 하드웨어 위에서 2번째 수준으로 실행된다. Type2(hosted)는 일반 프로그램과 같이 호스트 운영 체제에서 실행되며 VM 내부에서 동작되는 게스트 운영 체제는 하드웨어에서 3번째 수준으로 실행된다. VMware, vBox, Parallels Desktop이 대표적으로 Type2이다. 이렇게 Hypervisor 기술을 사용하는 가상환경은 다음과 같은 시스템 구조를 가진다.

<img src="./res/ch01/fig01.png" width="300" height="300"><br>
<div align="center">
  Figure01
</div>

Hypervisor와 Guest OS까지 동작중에 그 위에서 Application이 동작한다는 관점에서 리소스 소모가 적지않다. 

### 3. 컨테이너
위의 내용과 반대로 컨테이너의 경우는 다음과 같은 시스템 구조를 가진다.

<img src="./res/ch01/fig02.png" width="300" height="300"><br>
<div align="center">
  Figure02
</div>

즉, Hypervisor 기술을 사용하는 서비스의 단점인 Hypervisor위에서 Guest OS가 돌아가고, 그 위에서 App이 돌아가는 무거운 상황을 없애고, Container engine만 두고, 그 위에서 여러가지 환경에 맞는 App을 실행하는 모델이다. 이를 실현하는 방법은 기존 리눅스에서 제공되는 2가지 서비스를 이용한다.

### 4. 컨테이너의 원리
#### 4.1. cgroups
cgroups(control groups)는 프로세스들의 자원의 사용(CPU, Memory, Disk IO, Network and so on)을 제한하고 격리시키는 리눅스 커널 기능이다. 코어OS, 도커, 하둡 등 많은 프로젝트들이 채택하였다. 

#### 4.2. Linux Namespace
Namespace는 한 덩어리의 데이터에 이름을 붙여 충돌 가능성을 줄이고 쉽게 참조할 수 있게한다. Linux kernel의 namespace 기능은 linux의 오브젝트에 이름을 붙임으로써 다음과 같은 6개의 독립된 환경을 구축할 수 있다. 

1. PID namespace : <br>
    프로세스에 할당된 고유한 ID를 이용하여 프로세스를 격리할 수 있다. 이를 통하여 namespace가 다른 프로세스끼리는 서로 액세스할 수 없다.
2. Network namespace : <br>
    네트워크 디바이스, IP 주소, Port 번호, 라우팅 테이블, 필터링 테이블 등의 네트워크 리소스를 namespace마다 격리시켜 독립적으로 가질 수 있다. 이 기능을 통해서 OS 상에서 사용중인 Port가 있더라도 컨테이너 안에서 동일한 Port를 사용 가능하다.
3. UID namespace : <br>
    UID, GID를 namespace 별로 독립적으로 가질 수 있도록한다.
4. Mount namespace : <br>
    호스트 OS와 namespace가 서로 다른 격리된 파일시스템 트리를 가질 수 있도록 한다. 
5. UTS namespace : <br>
    namespace 별로 호스트명이나 도메인 명을 독자적으로 가질 수 있다.
6. IPC namespace : <br>
    프로세스간 통신(IPC) 오브젝트를 namespace 별로 독립적으로 가질 수 있다.

<br><br>
이렇게 cgroups과 namespace를 이용하여 도커 컨테이너처럼 가상화를 하여 프로그램을 실행할 수 있다. 이때, Hypervisor를 사용할 필요없으니 무겁게 Hypervisor위에 OS를 돌리고, 그 위에 프로세스를 돌릴필요가 없어진다. 추가적으로 Windows에서 도커를 사용하는 것은 공식적으로 지원을 하지만, Docker Host OS가 될 수 없다. 기본적으로 위에서 보았던 cgroups과 namespace는 linux 위에서 돌아가는 것이고, Windows에서 도커를 이용하면 Hypervisor를 사용하기 떄문에 도커 컨테이너를 사용하는 의미가 없어진다. 

### 5. 성능의 차이

<img src="./res/ch01/fig03.png" width="350" height="300"><br>
<div align="center">
  Figure03
</div>

<img src="./res/ch01/fig04.png" width="600" height="350"><br>
<div align="center">
  Figure04
</div>

<img src="./res/ch01/fig05.png" width="300" height="600"><br>
<div align="center">
  Figure05
</div>

<img src="./res/ch01/fig06.png" width="300" height="600"><br>
<div align="center">
  Figure06
</div>

<strong>Reference.</strong><br>
https://ko.wikipedia.org/wiki/하이퍼바이저<br>
https://www.tayfundeger.com/docker-mi-vmware-mi.html<br>
Qi Zhang - A Comparative Study of Containers and Virtual Machines in Big Data Environment